In [1]:
!pip install -U langchain langchain-community langgraph langchain-openai

# BridgeDB API Integration with LangChain Agent

This notebook demonstrates the integration of the BridgeDB API with a LangChain-powered AI agent for biological identifier mapping.

## Table of Contents
1. Setup and Imports
2. BridgeDB API Wrapper
3. Custom LangChain Tool
4. AI Agent Configuration
5. Testing the Agent
6. Additional Utility Functions
7. Direct API Calls

## 1. Setup and Imports

First, let's set up our environment and import the necessary libraries.

In [2]:
# Import statements
import os
import requests
import pandas as pd
from dotenv import load_dotenv
from langchain_community.tools import BaseTool
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

# Load environment variables
load_dotenv()

# Set OpenAI API Key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OpenAI API Key is not set. Please check your .env file.")

os.environ["OPENAI_API_KEY"] = api_key

## 2. BridgeDB API Wrapper

We'll create a wrapper class for the BridgeDB API to handle identifier mapping requests.

In [3]:
# BridgeDbAPI Class
# This class encapsulates the functionality to interact with the BridgeDb web service:
# - `BASE_URL`: The base URL for the BridgeDb API
# - `map_identifier`: A method to map identifiers between different biological databases

class BridgeDbAPI:
    BASE_URL = "https://webservice.bridgedb.org"

    @staticmethod
    def map_identifier(species, source_ds, identifier):
        url = f"{BridgeDbAPI.BASE_URL}/{species}/xrefs/{source_ds}/{identifier}"
        response = requests.get(url)
        if response.status_code == 200:
            return [line.split('\t') for line in response.text.strip().split('\n')]
        else:
            return f"Error: {response.status_code} - {response.reason}"

    @staticmethod
    def get_supported_data_sources(species):
        url = f"{BridgeDbAPI.BASE_URL}/{species}/sourceDataSources"
        response = requests.get(url)
        if response.status_code == 200:
            return response.text.strip().split('\n')
        else:
            return f"Error: {response.status_code} - {response.reason}"

In [14]:
"""# IdentifierMappingTool Class
This class extends BaseTool to create a custom tool for Langchain:
- It parses queries and uses BridgeDBAPI to perform identifier mapping
- The `_run` method handles synchronous operations
- The `_arun` method is a placeholder for asynchronous operations (not implemented)

The `tools` list is created with an instance of this tool."""


class IdentifierMappingTool(BaseTool):
    name = "identifier_mapping"
    description = "Useful for mapping identifiers between different biological databases"

    def _run(self, query: str) -> str:
        # Parse the query; now expecting only three parts: species, source_ds, identifier
        parts = query.split(",")
        if len(parts) != 3:
            return "Error: Query should be in the format 'species, source_ds, identifier'"
        species, source_ds, identifier = [p.strip() for p in parts]
        
        # Attempt to map identifiers and handle potential errors
        try:
            result = BridgeDbAPI().map_identifier(species, source_ds, identifier)
            if isinstance(result, list):
                # Formatting the output as a list
                formatted_result = "\n".join([f"- {item}" for item in result])
                return f"Mapped identifiers:\n{formatted_result}"
            else:
                return f"Error: {result}"  # This handles non-list responses like error messages
        except Exception as e:
            return f"Error: {str(e)}"

    def _arun(self, query: str) -> str:
        # Async implementation (not needed for this tool)
        raise NotImplementedError("This tool does not support async")
    
tools = [IdentifierMappingTool()]

In [15]:
# Test with a different ENSG identifier (e.g., BRCA2)
tool = IdentifierMappingTool()
print(tool._run("Homo sapiens, Ensembl, ENSG00000139618"))

Mapped identifiers:
- ['214727_PM_at', 'Affy']
- ['GO:0006978', 'GeneOntology']
- ['GO:0072089', 'GeneOntology']
- ['GO:1990426', 'GeneOntology']
- ['11740416_at', 'Affy']
- ['GO:0006974', 'GeneOntology']
- ['uc058wfw.1', 'UCSC Genome Browser']
- ['GO:0043009', 'GeneOntology']
- ['BRCA2', 'HGNC']
- ['A0A590UJ24', 'Uniprot-TrEMBL']
- ['1N0W', 'PDB']
- ['GO:0032465', 'GeneOntology']
- ['3484692', 'Affy']
- ['GO:0001833', 'GeneOntology']
- ['GO:0045893', 'GeneOntology']
- ['3484695', 'Affy']
- ['3484694', 'Affy']
- ['GO:0001556', 'GeneOntology']
- ['3484697', 'Affy']
- ['GO:0005515', 'GeneOntology']
- ['3484696', 'Affy']
- ['3484699', 'Affy']
- ['3484698', 'Affy']
- ['208368_PM_s_at', 'Affy']
- ['GO:0005634', 'GeneOntology']
- ['675', 'Entrez Gene']
- ['GO:0006289', 'GeneOntology']
- ['GO:0003697', 'GeneOntology']
- ['GO:0045931', 'GeneOntology']
- ['GO:2000001', 'GeneOntology']
- ['7968484', 'Affy']
- ['GO:0007420', 'GeneOntology']
- ['GO:0008630', 'GeneOntology']
- ['GO:0032991', 'GeneO

## 3. Custom LangChain Tool

Now, let's create a custom LangChain tool that uses our BridgeDbAPI wrapper.

In [19]:
import requests

class IdentifierMappingTool(BaseTool):
    name = "identifier_mapping"
    description = """Map identifiers between different biological databases. 
    Input can be in the following formats:
    1. 'species, source_ds, identifier' (e.g., 'Homo sapiens, Ensembl, ENSG00000139618')
    2. 'source_ds, identifier' (e.g., 'Cpc, 2478')
    3. Just the identifier or gene/chemical name (e.g., 'ENSG00000139618' or 'BRCA2' or 'Busulfan')"""

    def _run(self, query: str) -> str:
        parts = [p.strip() for p in query.split(",")]
        
        if len(parts) == 1:
            return self._flexible_mapping(parts[0])
        elif len(parts) == 2:
            source_ds, identifier = parts
            return self._map_identifier("Human", source_ds, identifier)
        elif len(parts) == 3:
            species, source_ds, identifier = parts
            return self._map_identifier(species, source_ds, identifier)
        else:
            return "Error: Invalid query format."

    def _flexible_mapping(self, query):
        # Try mapping as Ensembl gene ID
        result = self._map_identifier("Human", "En", query)
        if "Error" not in result:
            return result

        # Try mapping as HGNC ID
        if query.startswith("HGNC:"):
            result = self._map_identifier("Human", "H", query)
            if "Error" not in result:
                return result

        # Try mapping as gene symbol
        result = self._map_identifier("Human", "H", query)
        if "Error" not in result:
            return result

        # Try mapping as PubChem compound ID
        result = self._map_identifier("Human", "Cpc", query)
        if "Error" not in result:
            return result

        # If it fails, assume it's a chemical name and try to find its PubChem ID
        pubchem_id = self._get_pubchem_id(query)
        if pubchem_id:
            return self._map_identifier("Human", "Cpc", pubchem_id)
        
        return f"Error: Unable to map identifier or find compound: {query}"

    def _map_identifier(self, species, source_ds, identifier):
        url = f"https://bridgedb.cloud.vhp4safety.nl/{species}/xrefs/{source_ds}/{identifier}"
        response = requests.get(url)
        if response.status_code == 200:
            mappings = [line.split('\t') for line in response.text.strip().split('\n')]
            if mappings:
                formatted_result = "\n".join([f"- {item[0]}\t{item[1]}" for item in mappings])
                return f"Mapped identifiers for {identifier} from {source_ds}:\n{formatted_result}"
            else:
                return f"No mappings found for {identifier} from {source_ds}"
        else:
            return f"Error: Failed to retrieve mappings. Status code: {response.status_code}"

    def _get_pubchem_id(self, chemical_name):
        url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{chemical_name}/cids/TXT"
        response = requests.get(url)
        if response.status_code == 200:
            return response.text.strip()
        return None

    async def _arun(self, query: str) -> str:
        return self._run(query)

## 4. AI Agent Configuration

Let's set up our AI agent using the custom tool we created.

In [12]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage

def create_bridgedb_agent():
    model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    tools = [IdentifierMappingTool()]
    memory = MemorySaver()
    agent_executor = create_react_agent(model, tools, checkpointer=memory)
    return agent_executor

bridgedb_agent = create_bridgedb_agent()

def run_agent_query(query):
    config = {"configurable": {"thread_id": "bridgedb_conversation"}}
    
    print(f"Query: {query}\n")
    for chunk in bridgedb_agent.stream(
        {"messages": [HumanMessage(content=query)]},
        config
    ):
        if 'agent' in chunk and 'messages' in chunk['agent']:
            for message in chunk['agent']['messages']:
                if hasattr(message, 'content') and message.content:
                    print(message.content)
        print("----")
    print("\n")

# Test queries

run_agent_query("What are the mappings for the compound with PubChem ID 2478?")
run_agent_query("Find mappings for Busulfan")
run_agent_query("Find mappings for Aflatoxin B1")

Query: What are the mappings for the compound with PubChem ID 2478?

----
----
The compound with PubChem ID 2478 has the following mappings across different databases:

- **ChemSpider**: 2384
- **EPA CompTox**: DTXSID3020910
- **HMDB**: HMDB15143
- **ChEBI**: 28901
- **PubChem**: 2478
- **Wikidata**: Q348922
- **CAS**: 55-98-1
- **DrugBank**: DB01008
- **ChEBI**: CHEBI:28901
- **Guide to Pharmacology**: 7136
- **KEGG Drug**: D00248
- **InChIKey**: COVZYZSDYWQREU-UHFFFAOYSA-N
- **HMDB**: HMDB0015143
- **ChEMBL compound**: CHEMBL820
- **KEGG Compound**: C06862

These identifiers can be used to find more information about the compound in various biological and chemical databases.
----


Query: Find mappings for Busulfan

----
----
----
----
----
----
----
----
----
----
----
----
The compound Busulfan has the following mappings across different databases:

- **ChemSpider**: 2384
- **EPA CompTox**: DTXSID3020910
- **HMDB**: HMDB15143
- **ChEBI**: 28901
- **PubChem**: 2478
- **Wikidata**: Q

In [17]:
def create_bridgedb_agent():
    model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    tools = [IdentifierMappingTool()]
    memory = MemorySaver()
    agent_executor = create_react_agent(model, tools, checkpointer=memory)
    return agent_executor

bridgedb_agent = create_bridgedb_agent()

# Test queries
run_agent_query("Map the gene ENSG00000139618 from Ensembl to other databases")
run_agent_query("What are the mappings for the compound with PubChem ID 2478?")
run_agent_query("Find mappings for Busulfan")

Query: Map the gene ENSG00000139618 from Ensembl to other databases

----
----
----
----
The gene **ENSG00000139618** from Ensembl has been mapped to various identifiers in different biological databases. Here are some of the mappings:

- **HGNC**: BRCA2
- **Entrez Gene**: 675
- **UniProt-TrEMBL**: A0A590UJ24, P51587, A0A590UJI7, A0A7P0T9D7, H0YE37, A0A590UJU6, H0YD86, A0A7P0TAP7
- **RefSeq**: NP_000050, NM_000059
- **Gene Ontology (GO)**: Multiple entries including GO:0006978, GO:0072089, GO:1990426, etc.
- **Affymetrix**: 214727_PM_at, 11740416_at, 3484692, etc.
- **UCSC Genome Browser**: uc058wfw.1, uc058wfv.2, uc288ccz.1, etc.
- **PDB**: 1N0W, 7LDG, 6GY2, 3EU7, 6HQU
- **OMIM**: 600185
- **Wikidata**: Q17853272

This gene is associated with various biological functions and pathways, as indicated by the numerous Gene Ontology terms. If you need more specific information or additional mappings, feel free to ask!
----


Query: What are the mappings for the compound with PubChem ID 2478

## 5. Testing the Agent

Now let's test our agent with some example queries.

In [20]:
def create_bridgedb_agent():
    model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    tools = [IdentifierMappingTool()]
    memory = MemorySaver()
    agent_executor = create_react_agent(model, tools, checkpointer=memory)
    return agent_executor

bridgedb_agent = create_bridgedb_agent()

# Test queries
print("Test 1: Gene Identifier Queries\n")
run_agent_query("Map the gene ENSG00000130203 from Ensembl to other databases")
run_agent_query("What are the mappings for the HGNC gene ID HGNC:1101?")
run_agent_query("Find identifiers for the human gene BRCA2")

Test 1: Gene Identifier Queries

Query: Map the gene ENSG00000130203 from Ensembl to other databases

----
----
The gene **ENSG00000130203** from Ensembl has been mapped to various identifiers in different biological databases. Here are some of the mappings:

### Gene Ontology (GO)
- GO:0043083
- GO:0016020
- GO:0043524
- GO:0042311
- GO:0046911
- GO:0034364
- GO:0034365
- GO:0034362
- GO:0043407
- GO:0034363
- GO:0120009
- GO:0050728
- GO:0071682
- GO:0034380
- GO:1902952
- GO:0090181
- GO:0034375
- GO:0034374
- GO:0062023
- GO:0034372
- GO:0046907
- GO:0097006
- GO:0097487
- GO:0044877
- GO:0001540
- GO:0005102
- GO:1902995
- GO:0034384
- GO:0034382
- GO:0055088
- GO:1902991
- GO:0055089
- GO:0070328
- GO:0120020
- GO:0051044
- GO:0050750
- GO:0070326
- GO:0019068
- GO:0048662
- GO:0097113
- GO:0097114
- GO:0043537
- GO:0005576
- GO:0006979
- GO:0005769
- GO:0042159
- GO:0005886
- GO:0042158
- GO:0042157
- GO:0071830
- GO:0007271
- GO:1900272
- GO:0071813
- GO:0045893
- GO:0046983
- 

In [21]:
#2. Compound identifier mappings
print("Test 2: Compound Queries by ID\n")
run_agent_query("What are the mappings for the compound with PubChem ID 2478?")
run_agent_query("Map the ChEBI ID CHEBI:28901 to other databases")
run_agent_query("Find identifiers for the DrugBank entry DB01008")

Test 2: Compound Queries by ID

Query: What are the mappings for the compound with PubChem ID 2478?

----
----
The compound with **PubChem ID 2478** has been mapped to various identifiers across different databases. Here are the mappings:

### Compound Identifiers
- **PubChem**: 2478
- **CAS**: 55-98-1
- **ChEBI**: 28901
- **HMDB**: HMDB15143
- **DrugBank**: DB01008
- **KEGG Drug**: D00248
- **KEGG Compound**: C06862
- **ChemSpider**: 2384
- **ChEMBL**: CHEMBL820
- **EPA CompTox**: DTXSID3020910
- **Guide to Pharmacology**: 7136
- **Wikidata**: Q348922
- **InChIKey**: COVZYZSDYWQREU-UHFFFAOYSA-N
- **HMDB**: HMDB0015143

This mapping provides a comprehensive overview of the compound's identifiers across various biological and chemical databases.
----


Query: Map the ChEBI ID CHEBI:28901 to other databases

----
----
The ChEBI ID **CHEBI:28901** has been mapped to various identifiers across different databases. Here are the mappings:

### Compound Identifiers
- **ChEBI**: CHEBI:28901
- 

In [22]:
#3. Chemical name to identifier mappings
print("Test 3: Compound Queries by Name\n")
run_agent_query("Find mappings for Busulfan")
run_agent_query("What are the identifiers for Aflatoxin B1?")
run_agent_query("Map the drug Ibuprofen to different databases")

Test 3: Compound Queries by Name

Query: Find mappings for Busulfan

----
----
The compound **Busulfan** has been mapped to various identifiers across different databases. Here are the mappings:

### Busulfan Identifiers
- **PubChem**: 2478
- **CAS**: 55-98-1
- **ChEBI**: CHEBI:28901
- **HMDB**: HMDB15143
- **DrugBank**: DB01008
- **KEGG Drug**: D00248
- **KEGG Compound**: C06862
- **ChemSpider**: 2384
- **ChEMBL**: CHEMBL820
- **EPA CompTox**: DTXSID3020910
- **Guide to Pharmacology**: 7136
- **Wikidata**: Q348922
- **InChIKey**: COVZYZSDYWQREU-UHFFFAOYSA-N
- **HMDB**: HMDB0015143

This mapping provides a comprehensive overview of Busulfan's identifiers across various biological and chemical databases.
----


Query: What are the identifiers for Aflatoxin B1?

----
----
The compound **Aflatoxin B1** has been mapped to various identifiers across different databases. Here are the mappings:

### Aflatoxin B1 Identifiers
- **PubChem**: 186907
- **CAS**: 1162-65-8
- **ChEBI**: CHEBI:2504
- 

In [23]:
#4. More complex queries that require multiple steps=
print("Test 4: Edge Cases and Complex Queries\n")
run_agent_query("What's the HGNC symbol for ENSG00000139618?")
run_agent_query("Compare the identifiers for Busulfan and Ibuprofen")
run_agent_query("Find the PubChem ID for aspirin and then map it to other databases")
run_agent_query("Is there a connection between the gene BRCA2 and any known drugs?")

Test 4: Edge Cases and Complex Queries

Query: What's the HGNC symbol for ENSG00000139618?

----
----
The HGNC symbol for the Ensembl gene ID **ENSG00000139618** is **BRCA2**.
----


Query: Compare the identifiers for Busulfan and Ibuprofen

----
----
Here is a comparison of the identifiers for **Busulfan** and **Ibuprofen** across various databases:

### Busulfan Identifiers
- **PubChem**: 2478
- **CAS**: 55-98-1
- **ChEBI**: CHEBI:28901
- **HMDB**: HMDB15143
- **DrugBank**: DB01008
- **KEGG Drug**: D00248
- **KEGG Compound**: C06862
- **ChemSpider**: 2384
- **ChEMBL**: CHEMBL820
- **EPA CompTox**: DTXSID3020910
- **Wikidata**: Q348922
- **InChIKey**: COVZYZSDYWQREU-UHFFFAOYSA-N
- **HMDB**: HMDB0015143
- **Guide to Pharmacology**: 7136

### Ibuprofen Identifiers
- **PubChem**: 3672
- **CAS**: 15687-27-1
- **ChEBI**: CHEBI:5855
- **HMDB**: HMDB01925
- **DrugBank**: DB01050
- **KEGG Drug**: D00126
- **KEGG Compound**: C01588
- **ChemSpider**: 3544
- **ChEMBL**: CHEMBL521
- **EPA CompTox

In [24]:
#Edge cases and potentially problematic queries
print("Test 5: Incorrect or Ambiguous Queries\n")
run_agent_query("Map the identifier XYZ123")
run_agent_query("What are the mappings for water?")
run_agent_query("Find identifiers for COVID-19")
run_agent_query("Map ENSG00000139618 to PubChem")

Test 5: Incorrect or Ambiguous Queries

Query: Map the identifier XYZ123

----
----
It seems that the identifier **XYZ123** could not be mapped to any known entries in the biological or chemical databases. This may be due to it not being a recognized identifier or it may not correspond to any existing compound, gene, or protein.

If you have another identifier or a specific query in mind, please let me know, and I'll be happy to assist!
----


Query: What are the mappings for water?

----
----
The compound **Water** has been mapped to various identifiers across different databases. Here are the mappings:

### Water Identifiers
- **PubChem**: 962
- **CAS**: 7732-18-5
- **ChEBI**: CHEBI:15377
- **HMDB**: HMDB0002111
- **DrugBank**: DB09145
- **KEGG Drug**: D00001
- **KEGG Compound**: C00001
- **ChemSpider**: 937
- **ChEMBL**: CHEMBL1098659
- **EPA CompTox**: DTXSID6026296
- **Wikidata**: Q283
- **InChIKey**: XLYOFNOQVPJJNP-UHFFFAOYSA-N
- **HMDB**: HMDB02111

This mapping provides a compr

## 6. Additional Utility Functions

Here are some additional functions to work with the BridgeDB API directly.

In [ ]:
import requests
import pandas as pd

def map_identifier_to_df(species, source_ds, identifier):
    url = f"https://webservice.bridgedb.org/{species}/xrefs/{source_ds}/{identifier}"
    response = requests.get(url)
    if response.status_code != 200:
        return pd.DataFrame({'Error': [f"Failed with status code {response.status_code}"]})

    # Parsing the response
    rows = [line.split('\t') for line in response.text.split('\n') if line]
    df = pd.DataFrame(rows, columns=['Mapped Identifier', 'Target Database'])
    df['Original Identifier'] = identifier
    df['Source Database'] = source_ds
    
    # Reordering columns
    return df[['Original Identifier', 'Source Database', 'Mapped Identifier', 'Target Database']]

# Example usage
result_df = map_identifier_to_df("Homo sapiens", "Cpc", "186907")
result_df

,Original Identifier,Source Database,Mapped Identifier,Target Database
0,186907,Cpc,C00000546,KNApSAcK
1,186907,Cpc,CHEBI:2504,ChEBI
2,186907,Cpc,C06800,KEGG Compound
3,186907,Cpc,DTXSID00873175,EPA CompTox
4,186907,Cpc,186907,PubChem-compound
5,186907,Cpc,OQIQSTLJSLGHID-WNWIJWBNSA-N,InChIKey
6,186907,Cpc,CHEMBL1697694,ChEMBL compound
7,186907,Cpc,DTXSID9020035,EPA CompTox
8,186907,Cpc,1162-65-8,CAS
9,186907,Cpc,162470,Chemspider


In [ ]:
import requests
import pandas as pd

def map_batch_identifiers(species, source_ds, identifiers):
    url = f"https://webservice.bridgedb.org/{species}/xrefsBatch/{source_ds}"
    response = requests.post(url, data='\n'.join(identifiers))
    if response.status_code != 200:
        return pd.DataFrame({'Error': [f"Failed with status code {response.status_code}"]})

    # Parsing the response
    records = []
    for line in response.text.strip().split('\n'):
        if line:
            parts = line.split('\t', 1)
            if len(parts) == 2:
                original_id, mappings = parts
                for mapping in mappings.split(','):
                    if ':' in mapping:
                        target_db, mapped_id = mapping.split(':', 1)
                        records.append((original_id, source_ds, mapped_id, target_db))
                    else:
                        records.append((original_id, source_ds, 'Parsing Error', mapping))
            else:
                records.append((line, source_ds, 'Parsing Error', ''))

    df = pd.DataFrame(records, columns=['Original Identifier', 'Source Database', 'Mapped Identifier', 'Target Database'])
    
    if df.empty:
        return "No mappings found for the provided identifiers."
    
    return df

# Example usage
batch_result_df = map_batch_identifiers("Homo sapiens", "H", ["A1BG", "A1CF", "A2MP1"])
batch_result_df

,Original Identifier,Source Database,Mapped Identifier,Target Database
0,A1BG,H,3872844,HGNC\tX
1,A1BG,H,3872845,X
2,A1BG,H,3872842,X
3,A1BG,H,8039748,X
4,A1BG,H,HGNC:5,Hac
...,...,...,...,...
241,A2MP1,H,3443560,X
242,A2MP1,H,3443561,X
243,A2MP1,H,A_21_P0000904,Ag
244,A2MP1,H,3443562,X


In [ ]:
import requests
import pandas as pd

def get_supported_data_sources(species):
    url = f"https://webservice.bridgedb.org/{species}/sourceDataSources"
    response = requests.get(url)
    if response.status_code != 200:
        return pd.DataFrame({'Error': [f"Failed with status code {response.status_code}"]})

    # Parsing the response
    sources = response.text.split('\n')
    df = pd.DataFrame({'Source Databases': sources})
    return df

# Example usage
supported_sources_df = get_supported_data_sources("Homo sapiens")
supported_sources_df

,Source Databases
0,Rfam
1,Guide to Pharmacology
2,Rhea
3,ChEBI
4,HGNC
5,Ensembl
6,Wikidata
7,PubMed Central
8,Illumina
9,KEGG Drug


In [ ]:
import requests

# Example for Busulfan
busulfan_url = "https://bridgedb.cloud.vhp4safety.nl/Human/xrefs/Cpc/2478"
response = requests.get(busulfan_url)

if response.status_code == 200:
    print("Busulfan Mappings:")
    print(response.text)
else:
    print("Failed to retrieve Busulfan mappings:", response.status_code)

# Example for Aflatoxin B1
aflatoxin_url = "https://bridgedb.cloud.vhp4safety.nl/Human/xrefs/Cpc/186907"
response = requests.get(aflatoxin_url)

if response.status_code == 200:
    print("Aflatoxin B1 Mappings:")
    print(response.text)
else:
    print("Failed to retrieve Aflatoxin B1 mappings:", response.status_code)

Busulfan Mappings:
2384	Chemspider
DTXSID3020910	EPA CompTox
HMDB15143	HMDB
28901	ChEBI
2478	PubChem-compound
Q348922	Wikidata
55-98-1	CAS
DB01008	DrugBank
CHEBI:28901	ChEBI
7136	Guide to Pharmacology
D00248	KEGG Drug
COVZYZSDYWQREU-UHFFFAOYSA-N	InChIKey
HMDB0015143	HMDB
CHEMBL820	ChEMBL compound
C06862	KEGG Compound

Aflatoxin B1 Mappings:
C00000546	KNApSAcK
CHEBI:2504	ChEBI
C06800	KEGG Compound
DTXSID00873175	EPA CompTox
186907	PubChem-compound
OQIQSTLJSLGHID-WNWIJWBNSA-N	InChIKey
CHEMBL1697694	ChEMBL compound
DTXSID9020035	EPA CompTox
1162-65-8	CAS
162470	Chemspider
Q4689278	Wikidata
HMDB0006552	HMDB
2504	ChEBI
HMDB06552	HMDB

